In [2]:
from pyhive import hive

# Connect to Hive (adjust host/port as needed; here we're assuming the HiveServer is reachable on localhost:10000)

# Instead of host='localhost', use host='hive-server'
conn = hive.Connection(host='hive-server', port=10000, username='hive', database='default')
cursor = conn.cursor()

# Create the database if it doesn't exist (optional)
cursor.execute("CREATE DATABASE IF NOT EXISTS cancer_db")
cursor.execute("USE cancer_db")

# Drop the table if it exists to start fresh
cursor.execute("DROP TABLE IF EXISTS cancer_table")

# Create the table.
# Modify the schema below to match the columns in Cancer.csv.
cursor.execute("""
    CREATE TABLE cancer_table (
        patient_id STRING,
        age INT,
        gender STRING,
        air_pollution STRING,
        alcohol_use STRING,
        dust_allergy STRING,
        occupational_hazards STRING,
        genetic_risk STRING,
        chronic_lung_disease STRING,
        balanced_diet STRING,
        obesity STRING,
        smoking STRING,
        passive_smoker STRING,
        chest_pain STRING,
        coughing_of_blood STRING,
        fatigue STRING,
        weight_loss STRING,
        shortness_of_breath STRING,
        wheezing STRING,
        swallowing_difficulty STRING,
        clubbing_of_finger_nails STRING,
        frequent_cold STRING,
        dry_cough STRING,
        snoring STRING,
        level STRING
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
""")

# Load data into the table from the CSV file.
# The CSV file should be accessible at the specified path inside the Hive container.
cursor.execute("""
    LOAD DATA LOCAL INPATH '/data/Cancer.csv'
    OVERWRITE INTO TABLE cancer_table
""")

print("Data loaded successfully.")


Data loaded successfully.
